## Question 1: Install Spark and PySpark

In [1]:
import pyspark
import findspark
import pandas as pd
findspark.init()


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]") \
                    .appName('test-spark') \
                    .getOrCreate()

print(f'The PySpark {spark.version} version is running...')

25/02/26 15:00:02 WARN Utils: Your hostname, MacBook-Pro-Nikolay.local resolves to a loopback address: 127.0.0.1; using 192.168.0.150 instead (on interface en0)
25/02/26 15:00:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/26 15:00:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


The PySpark 3.5.4 version is running...


In [20]:
!ls

Untitled.ipynb                  yellow_tripdata_2024-10.parquet


In [46]:
from pyspark.sql import types

In [47]:
df.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

## Question 2: Yellow October 2024

In [15]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-02-25 16:53:22--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2759:ca00:b:20a5:b140:21, 2600:9000:2759:b000:b:20a5:b140:21, 2600:9000:2759:800:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:2759:ca00:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet.1’

yellow_tripdata_202 100%[===================>]  61.36M  31.9MB/s    in 1.9s    

2025-02-25 16:53:24 (31.9 MB/s) - ‘yellow_tripdata_2024-10.parquet.1’ saved [64346071/64346071]



Repartition the Dataframe to 4 partitions and save it to parquet.

In [5]:
df = spark.read\
.option("header", "true") \
.parquet("yellow_tripdata_2024-10.parquet")

In [7]:
df = df.repartition(4)
df.write.parquet('yellow/', mode = 'overwrite')

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

In [8]:
!ls -lh yellow

total 197120
-rw-r--r--  1 nikabramov  staff     0B Feb 26 15:01 _SUCCESS
-rw-r--r--  1 nikabramov  staff    24M Feb 26 15:01 part-00000-aedc014a-73c8-434a-89bc-4cc6ce144e0d-c000.snappy.parquet
-rw-r--r--  1 nikabramov  staff    24M Feb 26 15:01 part-00001-aedc014a-73c8-434a-89bc-4cc6ce144e0d-c000.snappy.parquet
-rw-r--r--  1 nikabramov  staff    24M Feb 26 15:01 part-00002-aedc014a-73c8-434a-89bc-4cc6ce144e0d-c000.snappy.parquet
-rw-r--r--  1 nikabramov  staff    24M Feb 26 15:01 part-00003-aedc014a-73c8-434a-89bc-4cc6ce144e0d-c000.snappy.parquet


## Question 3: Count records

How many taxi trips were there on the 15th of October?



In [9]:
df.registerTempTable('trips_data')

/Users/nikabramov/Library/Python/3.12/lib/python/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [10]:
spark.sql("""
SELECT
    CAST(tpep_pickup_datetime AS DATE), 
    COUNT(*)
FROM
    trips_data
WHERE 
    CAST(tpep_pickup_datetime AS DATE) = '2024-10-15'
    --AND fare_amount > 0
GROUP BY 1
ORDER BY 1
    
""").show()

+--------------------+--------+
|tpep_pickup_datetime|count(1)|
+--------------------+--------+
|          2024-10-15|  128893|
+--------------------+--------+



## Question 4: Longest trip

What is the length of the longest trip in the dataset in hours?

In [12]:
spark.sql("""
SELECT
    MAX(DATE_DIFF(HOUR, tpep_pickup_datetime, tpep_dropoff_datetime)) as longest_trip
FROM
    trips_data    
""").show()

+------------+
|longest_trip|
+------------+
|         162|
+------------+



## Question 5: User Interface

Spark’s User Interface which shows the application's dashboard runs on which local port?

- 4040

## Question 6: Least frequent pickup location zone

Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

In [67]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-02-26 14:51:31--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2759:3800:b:20a5:b140:21, 2600:9000:2759:3c00:b:20a5:b140:21, 2600:9000:2759:a00:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:2759:3800:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-02-26 14:51:31 (77.4 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [13]:
df_zone_lookup = spark.read \
    .option("header", "true")\
    .csv("taxi_zone_lookup.csv")

In [14]:
df_zone_lookup.createOrReplaceTempView("taxi_zone_lookup")


In [15]:
spark.sql("""
SELECT
    Zone
    , COUNT(*) as n_trips
FROM
    trips_data as t
LEFT JOIN
    taxi_zone_lookup as tzl
ON
    t.PULocationID = tzl.LocationID
GROUP BY 1
ORDER BY 2 ASC

""").show()

+--------------------+-------+
|                Zone|n_trips|
+--------------------+-------+
|Governor's Island...|      1|
|       Rikers Island|      2|
|       Arden Heights|      2|
|         Jamaica Bay|      3|
| Green-Wood Cemetery|      3|
|Charleston/Totten...|      4|
|   Rossville/Woodrow|      4|
|       West Brighton|      4|
|       Port Richmond|      4|
|Eltingville/Annad...|      4|
|         Great Kills|      6|
|        Crotona Park|      6|
|Heartland Village...|      7|
|     Mariners Harbor|      7|
|Saint George/New ...|      9|
|             Oakwood|      9|
|       Broad Channel|     10|
|New Dorp/Midland ...|     10|
|         Westerleigh|     12|
|     Pelham Bay Park|     12|
+--------------------+-------+
only showing top 20 rows

